###  Installation des bibliothèques : 

In [1]:
!pip install tweepy
!pip install networkx
!pip install textblob

### Import des bibliothèques : 

In [6]:
import pandas as pd 
import tweepy

###  Accés à l'environnement twitter : 

In [19]:

#### On remplace ici les valeurs des Key, secret et Token par celle de l'application crée sur l'environnement twitter
CONSUMER_KEY = 'MY_CONSUMER_KEY'
CONSUMER_SECRET = 'MY_CONSUMER_SECRET'
OAUTH_TOKEN = 'MY_OAUTH_TOKEN'
OAUTH_TOKEN_SECRET = 'MY_OAUTH_TOKEN_SECRET'


##### Connecxion à l'API
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

#  Token and secret
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

# Création de l'API
api = tweepy.API(auth) 

# # Afficher l'objet twitter API

api

### Recherche des publications liées à la publication déces du président Jacques Chirac :

In [9]:
# Creating the API object while passing in auth information
api = tweepy.API(auth)

# Le mots a cherché
query = "JacquesChirac"
# Langue
language = "fr"

# Calling the user_timeline function with our parameters
results = api.search_tweets(q=query, lang=language)

# foreach through all tweets pulled
for tweet in results:
   # printing the text stored inside the tweet object
   print(tweet.user.screen_name,"Tweeted:",tweet.text)

lagnose1 Tweeted: RT @djoko_officiel: @AMGKodjo @RFIAfrique @AmbUETogo @francediplo Le G5 est composé des PAYS qui ont asservi l' #AFRIQUEsans vergogne penda…
djoko_officiel Tweeted: @AMGKodjo @RFIAfrique @AmbUETogo @francediplo Le G5 est composé des PAYS qui ont asservi l' #AFRIQUEsans vergogne p… https://t.co/iXPqnFdLuY
yayabrunet Tweeted: RT @BensaadiSaid: Toute la chaîne médiatique est pro-israélienne. Les politiques ne veulent pas se la mettre à dos. Tour le monde ne s'appe…
BensaadiSaid Tweeted: Toute la chaîne médiatique est pro-israélienne. Les politiques ne veulent pas se la mettre à dos. Tour le monde ne… https://t.co/6MnY0FgTF3
BuonoRosy2 Tweeted: RT @Saurel_P: 10 mai 2006, commémoration de l'abolition de l'esclavage instituée par #JacquesChirac.
Si le nom de Victor Schoelcher est écr…
Majdoul34 Tweeted: RT @Saurel_P: 10 mai 2006, commémoration de l'abolition de l'esclavage instituée par #JacquesChirac.
Si le nom de Victor Schoelcher est écr…
MidiCitoyens Tweeted: RT @Saurel

###  Création de la dataframe : 

In [11]:
# intialize the dataframe
tweets_df = pd.DataFrame()
# populate the dataframe
for tweet in results:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)
# show the dataframe
tweets_df.head()

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,LAGNOSE,,"Philosophe, Spiritualiste,\n...Sincérité_Justice",False,2022-05-14 09:20:31+00:00,RT @djoko_officiel: @AMGKodjo @RFIAfrique @Amb...,[AFRIQUEsans],Twitter for Android
1,Djoko Christian Officiel,,Mon combat a pour but la Justice Pour Tous.\nN...,False,2022-05-14 09:15:18+00:00,@AMGKodjo @RFIAfrique @AmbUETogo @francediplo ...,[AFRIQUEsans],Twitter for Android
2,Chocolatstrychnine,chantonnay 85110,,False,2022-05-12 04:59:21+00:00,RT @BensaadiSaid: Toute la chaîne médiatique e...,None,Twitter for Android
3,Bensaadi Saïd,Auvergne Rhône-Alpes,"Je suis juste un citoyen de gauche, horrifié p...",False,2022-05-12 04:33:32+00:00,Toute la chaîne médiatique est pro-israélienne...,None,Twitter for Android
4,Buono Rosy,"Lattes, France",Ex Vice Présidente Métropole de Montpellier - ...,False,2022-05-11 13:46:17+00:00,"RT @Saurel_P: 10 mai 2006, commémoration de l'...",[JacquesChirac],Twitter for iPhone


### Sauvegare du fichier Json pour la création de la base de données Mongodb:

In [12]:
tweets_df.to_json(path_or_buf='publications.json', orient='records')

In [16]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   user_name         15 non-null     object             
 1   user_location     15 non-null     object             
 2   user_description  15 non-null     object             
 3   user_verified     15 non-null     bool               
 4   date              15 non-null     datetime64[ns, UTC]
 5   text              15 non-null     object             
 6   hashtags          13 non-null     object             
 7   source            15 non-null     object             
dtypes: bool(1), datetime64[ns, UTC](1), object(6)
memory usage: 983.0+ bytes
